In [1]:
import transformers
from transformers import LlamaForCausalLM, LlamaTokenizer
import sentencepiece
import torch

c:\Users\Shivam\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from accelerate import infer_auto_device_map, init_empty_weights
from transformers import AutoConfig, AutoModelForCausalLM

config = AutoConfig.from_pretrained("D:\ChatLLAMA")
with init_empty_weights():
    model = AutoModelForCausalLM.from_config(config)

In [3]:
from accelerate.utils import get_balanced_memory
# device_map = infer_auto_device_map(model, no_split_module_classes=["BloomBlock", "OPTDecoderLayer", "LLaMADecoderLayer", "LlamaDecoderLayer"], dtype="float16")
# max_memory = get_balanced_memory(
#     model,
#     max_memory=None,
#     no_split_module_classes=["BloomBlock", "OPTDecoderLayer", "LLaMADecoderLayer", "LlamaDecoderLayer"],
#     dtype='float16',
#     low_zero=False,
# )

device_map = infer_auto_device_map(
    model,
    # max_memory=max_memory,
    no_split_module_classes=["BloomBlock", "OPTDecoderLayer", "LLaMADecoderLayer", "LlamaDecoderLayer"],
    dtype='float16'
)
print(device_map)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


{'model.embed_tokens': 0, 'model.layers.0': 0, 'model.layers.1': 0, 'model.layers.2': 0, 'model.layers.3': 0, 'model.layers.4': 0, 'model.layers.5': 0, 'model.layers.6': 'cpu', 'model.layers.7': 'cpu', 'model.layers.8': 'cpu', 'model.layers.9': 'cpu', 'model.layers.10': 'cpu', 'model.layers.11': 'disk', 'model.layers.12': 'disk', 'model.layers.13': 'disk', 'model.layers.14': 'disk', 'model.layers.15': 'disk', 'model.layers.16': 'disk', 'model.layers.17': 'disk', 'model.layers.18': 'disk', 'model.layers.19': 'disk', 'model.layers.20': 'disk', 'model.layers.21': 'disk', 'model.layers.22': 'disk', 'model.layers.23': 'disk', 'model.layers.24': 'disk', 'model.layers.25': 'disk', 'model.layers.26': 'disk', 'model.layers.27': 'disk', 'model.layers.28': 'disk', 'model.layers.29': 'disk', 'model.layers.30': 'disk', 'model.layers.31': 'disk', 'model.norm': 'disk', 'lm_head': 'disk'}


In [4]:

tokenizer = LlamaTokenizer.from_pretrained("D:\ChatLLAMA")
device_map["model.layers.10"] = "disk"
# device_map["model.layers.9"] = "disk"

In [5]:
device_map

{'model.embed_tokens': 0,
 'model.layers.0': 0,
 'model.layers.1': 0,
 'model.layers.2': 0,
 'model.layers.3': 0,
 'model.layers.4': 0,
 'model.layers.5': 0,
 'model.layers.6': 'cpu',
 'model.layers.7': 'cpu',
 'model.layers.8': 'cpu',
 'model.layers.9': 'cpu',
 'model.layers.10': 'disk',
 'model.layers.11': 'disk',
 'model.layers.12': 'disk',
 'model.layers.13': 'disk',
 'model.layers.14': 'disk',
 'model.layers.15': 'disk',
 'model.layers.16': 'disk',
 'model.layers.17': 'disk',
 'model.layers.18': 'disk',
 'model.layers.19': 'disk',
 'model.layers.20': 'disk',
 'model.layers.21': 'disk',
 'model.layers.22': 'disk',
 'model.layers.23': 'disk',
 'model.layers.24': 'disk',
 'model.layers.25': 'disk',
 'model.layers.26': 'disk',
 'model.layers.27': 'disk',
 'model.layers.28': 'disk',
 'model.layers.29': 'disk',
 'model.layers.30': 'disk',
 'model.layers.31': 'disk',
 'model.norm': 'disk',
 'lm_head': 'disk'}

In [6]:
model = LlamaForCausalLM.from_pretrained("D:\ChatLLAMA", device_map=device_map,torch_dtype=torch.float16, offload_state_dict = True,offload_folder="D:\ChatLLAMA" )
# torch_dtype=torch.float16,offload_folder="D:\ChatLLAMA2"

Loading checkpoint shards: 100%|██████████| 2/2 [09:56<00:00, 298.41s/it]


In [ ]:
# stop_token_ids = [
#     tokenizer.convert_tokens_to_ids(x) for x in [
#         [''], ['User', ':'], ['system', ':'],
#         [tokenizer.convert_ids_to_tokens([9427])[0], ':']
#     ]
# ]

In [ ]:
# stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
# stop_token_ids
     

In [ ]:
# from transformers import StoppingCriteria, StoppingCriteriaList

# # define custom stopping criteria object
# class StopOnTokens(StoppingCriteria):
#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         for stop_ids in stop_token_ids:
#             if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():
#                 return True
#         return False

# stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
# generate_text = transformers.pipeline(
#     model=model, tokenizer=tokenizer,
#     return_full_text=True,  # langchain expects the full text
#     task='text-generation',
#     device=device,
#     # we pass model parameters here too
#     stopping_criteria=stopping_criteria,  # without this model will ramble
#     temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
#     top_p=0.15,  # select from top tokens whose probability add up to 15%
#     top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
#     max_new_tokens=256,  # max number of tokens to generate in the output
#     repetition_penalty=1.2  # without this output begins repeating
# )

In [10]:
prompt = 'Q: What is the capital of USA?\nA:'
# input_ids = torch.Tensor(tokenizer(prompt, return_tensors="pt").input_ids).clone().detach().cuda()
input_ids = tokenizer(prompt, return_tensors="pt").input_ids



In [8]:
model.eval()

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [11]:
generation_output = model.generate(
    input_ids=input_ids, max_new_tokens=20
)
print(tokenizer.decode(generation_output[0]))

<s>Q: What is the capital of USA?
A: Washington D.C.
Q: What is the currency of USA?
A: US D
